#Data

In [ ]:
month = list(range(1, 13))
for m in tqdm(month):
  m_geo = pd.read_parquet(f"/content/drive/MyDrive/mipt_2024/geo_train.parquet/mon={m}")
  cols = m_geo.columns.values.tolist()
  m_geo["event_time"] = pd.to_datetime(m_geo["event_time"].dt.strftime('%Y-%m'))
  m_geo = m_geo.groupby(['client_id','event_time']).agg(['mean']).reset_index()
  m_geo.columns = cols
  m_geo.to_csv(f"/content/drive/MyDrive/mipt_2024/geo_train_csv/m{m}.csv")
  del m_geo
  gc.collect()

  0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
month = list(range(1, 13))
for m in tqdm(month):
  m_dial = pd.read_parquet(f"/content/drive/MyDrive/mipt_2024/dial_train.parquet/mon={m}")
  embs = pd.DataFrame([a.tolist() for a in m_dial["embedding"].values.tolist()], columns=[f"d{i}" for i in range(len(test[0]))])
  m_dial = pd.concat([m_dial, embs], axis=1)
  m_dial = m_dial.drop(columns=["embedding"])
  cols = m_dial.columns
  m_dial["event_time"] = pd.to_datetime(m_dial["event_time"].dt.strftime('%Y-%m'))
  m_dial = m_dial.groupby(['client_id','event_time']).agg(['mean']).reset_index()
  m_dial.columns = cols
  m_dial.to_csv(f"/content/drive/MyDrive/mipt_2024/dial_train_csv/m{m}.csv")
  del m_dial
  gc.collect()

  0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
month = list(range(1, 13))
for m in tqdm(month):
  m_trx = pd.read_parquet(f"/content/drive/MyDrive/mipt_2024/trx_train.parquet/mon={m}")
  m_trx = m_trx.drop(columns=["event_type", "event_subtype", "currency"])
  cols = ['client_id', 'event_time', 'amount', 'src_type11', 'src_type12',
       'dst_type11', 'dst_type12', 'src_type21', 'src_type22', 'src_type31',
       'src_type32']
  m_trx["event_time"] = pd.to_datetime(m_trx["event_time"].dt.strftime('%Y-%m'))
  m_trx = m_trx.groupby(['client_id','event_time']).agg(['mean']).reset_index()
  m_trx.columns = cols
  m_trx.to_csv(f"/content/drive/MyDrive/mipt_2024/trx_train_csv/m{m}.csv")
  del m_trx
  gc.collect()

  0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
month = list(range(1, 13))
dials = []
for m in tqdm(month):
  dial = pd.read_csv(f"/content/drive/MyDrive/mipt_2024/dial_train_csv/m{m}.csv")
  dial["event_time"] = pd.to_datetime(dial["event_time"]).dt.strftime('%m')
  dials.append(dial)
total_dial = pd.concat(dials, axis=0)
total_dial.to_csv("/content/drive/MyDrive/mipt_2024/dial_train.csv", index=False)

  0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
month = list(range(1, 13))
geos = []
for m in tqdm(month):
  geo = pd.read_csv(f"/content/drive/MyDrive/mipt_2024/geo_train_csv/m{m}.csv")
  geo["event_time"] = pd.to_datetime(geo["event_time"]).dt.strftime('%m')
  geos.append(geo)
total_geo = pd.concat(geos, axis=0)
total_geo.to_csv("/content/drive/MyDrive/mipt_2024/geo_train.csv", index=False)

  0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
month = list(range(1, 13))
trxs = []
for m in tqdm(month):
  trx = pd.read_csv(f"/content/drive/MyDrive/mipt_2024/trx_train_csv/m{m}.csv")
  trx["event_time"] = pd.to_datetime(trx["event_time"]).dt.strftime('%m')
  trxs.append(trx)
total_trx = pd.concat(trxs, axis=0)
total_trx.to_csv("/content/drive/MyDrive/mipt_2024/trx_train.csv", index=False)

  0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
targets = pd.read_parquet("/content/drive/MyDrive/mipt_2024/train_target.parquet")
targets["mon"] = pd.to_datetime(targets["mon"]).dt.strftime('%m')
targets = targets.groupby(['client_id','mon']).agg(['max']).reset_index()
targets.columns = ['client_id', 'event_time', 'target_1', 'target_2', 'target_3', 'target_4']
targets.to_csv("/content/drive/MyDrive/mipt_2024/target_train.csv")

In [ ]:
files = ["trx_train.csv", "dial_train.csv", "geo_train.csv", "target_train.csv"]
ids = []
for f in tqdm(files):
  data = pd.read_csv(f"/content/drive/MyDrive/mipt_2024/{f}")
  uid = data["client_id"].unique().tolist()
  ids = list(set(ids + uid))
  del data
  gc.collect()

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
ids = pd.DataFrame({"client_id":ids})
ids.to_csv("/content/drive/MyDrive/mipt_2024/ids.csv")

In [ ]:
num_patches = 10
num_patch_ids = int(len(ids)/num_patches)
patches = []
ind = 0
for i in range(num_patches):
  if i != num_patches - 1:
    patches.append(ids["client_id"].values.tolist()[ind: ind + num_patch_ids])
  else:
    patches.append(ids["client_id"].values.tolist()[ind:])
  ind += num_patch_ids

In [ ]:
for f in tqdm(files):
  data = pd.read_csv(f"/content/drive/MyDrive/mipt_2024/{f}")
  for i, p in tqdm(enumerate(patches), total=len(patches)):
    patch = data[data["client_id"].isin(p)]
    path = f"/content/drive/MyDrive/mipt_2024/{f.replace('.csv', f'_csv/p{i}.csv')}"
    patch.to_csv(path)
  del data

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
for i in tqdm(range(10)):
  dial_data = pd.read_csv(f"/content/drive/MyDrive/mipt_2024/dial_test_csv/p{i}.csv")
  geo_data = pd.read_csv(f"/content/drive/MyDrive/mipt_2024/geo_test_csv/p{i}.csv")
  trx_data = pd.read_csv(f"/content/drive/MyDrive/mipt_2024/trx_test_csv/p{i}.csv")
  target_data = pd.read_csv(f"/content/drive/MyDrive/mipt_2024/target_test_csv/p{i}.csv")

  dial_data = dial_data.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
  geo_data = geo_data.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
  trx_data = trx_data.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
  target_data = target_data.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])

  data = dial_data.merge(geo_data, how='left', on=['client_id', 'event_time']).merge(trx_data, how='left', on=['client_id', 'event_time']).merge(target_data, how='left', on=['client_id', 'event_time'])
  data.to_csv(f"/content/drive/MyDrive/mipt_2024/train_p{i}.csv")

#CLIP-like

In [ ]:
import pandas as pd
import torch
from transformers import Trainer
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.functional as F
from transformers.utils import ModelOutput
from tqdm.notebook import tqdm
import gc

In [ ]:
class CLIPDataset(Dataset):
  def __init__(self, config):
    self.data = pd.read_csv(config["data"])
    if config["train"]:
      self.data = self.data.iloc[0:int(len(self.data)*(1-config["val_size"]))]
    else:
      self.data = self.data.iloc[int(len(self.data)*(1-config["val_size"])):]
    self.dialogs = self.data[config["dialog_cols"]]
    self.trans = self.data[config["trans_cols"]]
    self.geo = self.data[config["geo_cols"]]
    self.targets = self.data[config["target_cols"]]
  def __len__(self):
    return len(self.data)
  def __getitem__(self, ind):
    dialog = self.dialogs.iloc[ind].values
    trans = self.trans.iloc[ind].values
    geo = self.geo.iloc[ind].values
    target = self.targets.iloc[ind].values
    sample = {
        "dialog":torch.from_numpy(dialog),
        "trans":torch.from_numpy(trans),
        "geo":torch.from_numpy(geo),
        "target":torch.from_numpy(target),
    }
    return sample

In [ ]:
class CLIPModel(nn.Module):
  def __init__(self, config):
    super().__init__()
    layers = [[], [], [], []]
    for i in range(config["n_layers"]):
      for l in layers:
        l.append(nn.Linear(config["hidden_dim"]))
    self.dialog_model = nn.Sequential(
        nn.Linear(config["dialog_dim"], config["hidden_dim"]),
        *layers[0],
        nn.Linear(config["hidden_dim"], config["emb_dim"])
    )
    self.trans_model = nn.Sequential(
        nn.Linear(config["trans_dim"], config["hidden_dim"]),
        *layers[1],
        nn.Linear(config["hidden_dim"], config["emb_dim"])
    )
    self.geo_model = nn.Sequential(
        nn.Linear(config["geo_dim"], config["hidden_dim"]),
        *layers[2],
        nn.Linear(config["hidden_dim"], config["emb_dim"])
    )
    self.target_model = nn.Sequential(
        nn.Linear(config["target_dim"], config["hidden_dim"]),
        *layers[3],
        nn.Linear(config["hidden_dim"], config["emb_dim"])
    )
  def forward(self, batch):
    embs = self.get_embs(batch)

    pairs = {}
    for key1 in batch.keys():
      for key2 in batch.keys():
        pair_key = f"{key1}_{key2}"
        pairs[pair_key] = [None, None]

    for pair_key in pairs.keys():
      k1, k2 = pair_key.split("_")
      for i in range(len(batch["dialog"])):
        if batch[k1][i].sum() != 0 and batch[k2].sum() != 0:
          if pairs[pair_key][0] == None:
            pairs[pair_key][0] = batch[k1][i][None, :]
            pairs[pair_key][0] = batch[k2][i][None, :]
          else:
            pairs[pair_key][0] = torch.cat([pairs[pair_key][0], batch[k1][i][None, :]], dim=0)
            pairs[pair_key][0] = torch.cat([pairs[pair_key][0], batch[k2][i][None, :]], dim=0)

    loss = 0
    for pair_key in pairs.keys():
      emb1, emb2 = pairs[pair_key]
      local_loss = self.pairvise_loss(emb1, emb2)
      locc += local_loss
    output = ModelOutput()
    output["loss"] = loss
    return output

  def inference(self, batch):
    embs = self.get_embs(batch)
    masks = {
        "dialog":torch.ones(embs["dialog"].shape[0]),
        "trans":torch.ones(embs["dialog"].shape[0]),
        "geo":torch.ones(embs["dialog"].shape[0]),
        "target":torch.ones(embs["dialog"].shape[0]),
    }
    for key in batch.keys():
      masks[key][batch[key].sum(1) == 0] = 0
    for key in batch.keys():
      embs[key] = embs[key]*masks[key]

    total_embs = torch.zeros_like(embs["dialog"])
    for key in batch.keys():
      total_embs = total_embs + embs[key]
    out = {
        "out":total_embs
    }

  def get_embs(self, batch):
    models = {
        "dialog":self.dialog_model,
        "trans":self.trans_model,
        "geo":self.geo_model,
        "target":self.target_model
    }
    embs = {
        "dialog":None,
        "trans":None,
        "geo":None,
        "target":None
    }
    for key in batch.keys():
      sample = batch[key]
      emb = models[key](sample)
      embs[key] = emb
    return embs

  def pairvise_loss(self, emb1, emb2):
      logits = emb1 @ emb2.T
      sim1 = emb1 @ emb1.T
      sim2 = emb2 @ emb2.T
      targets = F.softmax(
          sim1 + sim2, dim=-1
      )
      texts_loss = self.cross_entropy(logits, targets, reduction='mean')
      images_loss = self.cross_entropy(logits.T, targets.T, reduction='mean')
      loss =  (images_loss + texts_loss) / 2.0 # shape: (batch_size)

  def cross_entropy(self, preds, targets, reduction='none'):
      log_softmax = nn.LogSoftmax(dim=-1)
      loss = (-targets * log_softmax(preds)).sum(1)
      if reduction == "none":
          return loss
      elif reduction == "mean":
          return loss.mean()

In [ ]:
path = ""
data = pd.read_csv(path)
train_data_config = {
    "data":data,
    "dialog_cols":[f"d{i}" for i in range(768)],
    "trans_cols":['amount', 'src_type11', 'src_type12',
       'dst_type11', 'dst_type12', 'src_type21', 'src_type22', 'src_type31',
       'src_type32'],
    "geo_cols":["geohash_4", "geohash_5", "geohash_6"],
    "target_cols":["target_1", "target_2", "target_3", "target_4"],
    "train":True,
    "val_size":0.2,
}
val_data_config = {
    "data":data,
    "dialog_cols":[f"d{i}" for i in range(768)],
    "trans_cols":['amount', 'src_type11', 'src_type12',
       'dst_type11', 'dst_type12', 'src_type21', 'src_type22', 'src_type31',
       'src_type32'],
    "geo_cols":["geohash_4", "geohash_5", "geohash_6"],
    "target_cols":["target_1", "target_2", "target_3", "target_4"],
    "train":False,
    "val_size":0.2,
}
train_dataset = CLIPDataset(train_data_config)
val_dataset = CLIPDataset(val_data_config)

In [ ]:
model_config = {
    "dialog_dim":0,
    "trans_dim":0,
    "geo_dim":0,
    "target_dim":0,
    "emd_dim":128,
    "hidden_dim":128,
    "n_layers":4
}
clip_model = CLIPModel(model_config)

In [ ]:
batch_size = 16
num_workers = 4
training_args = TrainingArguments(
    output_dir="/content",
    overwrite_output_dir=True,
    # learning_rate=0.001,
    num_train_epochs=100,
    do_eval=True,
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    dataloader_num_workers=num_workers,
    save_strategy="epoch",
    logging_strategy="epoch",
    save_total_limit=3,
    logging_dir="/content",  # Make sure to specify a logging directory
    load_best_model_at_end=True,  # Load the best model when training ends
    metric_for_best_model="eval_loss",  # Metric to monitor for early stopping
    greater_is_better=False,  # For loss
)

# define trainer
trainer = Trainer(
    model=clip_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    # compute_metrics=compute_metrics,
)

# pretrain
trainer.train()


In [ ]:
model = trainer.model
for i in tqdm(range(3, 10)):
  device = "cuda" if torch.cuda.is_available() else "cpu"
  path = f"/content/drive/MyDrive/mipt_2024/test_p{i}.csv"
  data = pd.read_csv(path)
  data_config = {
      "data":data,
      "dialog_cols":[f"d{i}" for i in range(768)],
      "trans_cols":['amount', 'src_type11', 'src_type12',
        'dst_type11', 'dst_type12', 'src_type21', 'src_type22', 'src_type31',
        'src_type32'],
      "geo_cols":["geohash_4", "geohash_5", "geohash_6"],
      "target_cols":["target_1", "target_2", "target_3", "target_4"],
      "train":True,
      "val_size":0,
  }
  dataset = CLIPDataset(data_config)
  dataloader = DataLoader(dataset, batch_size=16, shuffle=False)

  embs = None
  for batch in tqdm(dataloader):
    for key in batch.keys():
      batch[key] = batch[key].to(device)
    out = model.inference(batch)["out"]
    if embs == None:
      embs = out
    else:
      embs = torch.cat([embs, out], dim=0)
  embs = pd.DataFrame(embs.tolist(), columns=[f"e{i}" for i in range(128)])
  data = pd.concat([data, embs], axis=1).drop(columns=["Unnamed: 0"])

  data.to_csv(f"embs_p{i}.csv")
  del data
  gc.collect()

#PatchTST

Architecture is presented in article [64 Words: Long-term Forecasting with Transformers ](https://huggingface.co/papers/2211.14730)

In [ ]:
class TSDataset(Dataset):
  def __init__(self, config):
    path = config["path"]
    data = pd.read_csv(path)
    data = data.sort_values(by="client_id")
    if config["train"]:
      data = data.iloc[0:int(len(data)*(1-config["val_size"]))]
    else:
      data = data.iloc[int(len(data)*(1-config["val_size"])):]
    ids = data["client_id"].unique().tolist()
    base_embs = []
    base_masks = []
    base_times = []
    for id in ids:
      id_data = data[data["client_id"] == id]
      embs_data = id_data[["event_time"] + config["emb_cols"]].sort_values("event_time")
      embs = embs_data[config["emb_cols"]].values.tolist()
      times = embs_data["event_time"].values.tolist()
      local_masks = [True]
      local_embs = [embs[-1]]
      local_times = [str(times[-1]).rjust(2, "0")]
      last_month = times[-1]
      ind = len(embs_data) - 2
      for step in range(1, config["past_size"] + 1):
        if ind < 0:
          local_masks.append(False)
          local_embs.append([0 for i in range(len(embs[-1]))])
          last_month = last_month - 1
          new_time = str(last_month).rjust(2, "0")
          local_times.append(new_time)
          continue
        month = int(times[ind])
        if last_month - month > 1:
          local_masks.append(False)
          local_embs.append([0 for i in range(len(embs[-1]))])
          last_month = last_month - 1
          new_time = str(last_month).rjust(2, "0")
          local_times.append(new_time)
        else:
          local_masks.append(True)
          local_embs.append(embs[ind])
          local_times.append(str(times[ind]).rjust(2, "0"))
          ind -= 1
          last_month = month

        """if len(embs_data) > 1:
          print(len(embs_data))
          print(local_embs)
          print(local_masks)
          print(local_times)"""
      base_embs.append([local_embs[i] for i in range(len(local_embs)-1, -1, -1)])
      base_masks.append([local_masks[i] for i in range(len(local_masks)-1, -1, -1)])
      base_times.append([local_times[i] for i in range(len(local_times)-1, -1, -1)])
      """if len(embs_data) > 1:
        print(base_embs)
        print(base_masks)
        print(base_times)"""
      self.embs = []
      self.masks = []
      self.times = []
      for i in range(len(base_embs)):
        valid = sum([1 if base_masks[i][j] == True else 0  for j in range(len(base_masks[i]))])
        if valid > 1:
          self.embs.append(base_embs[i])
          self.masks.append(base_masks[i])
          self.times.append(base_times[i])

  def __len__(self):
    return len(self.embs)
  def __getitem__(self, ind):
    emb = self.embs[ind]
    time = self.times[ind]
    mask = self.masks[ind]
    sample = {
        'past_values':torch.Tensor(emb[:-1]),
        'future_values':torch.Tensor([emb[-1]]),
        'past_observed_mask':torch.Tensor(mask[:-1]),
        'future_observed_mask':torch.Tensor([mask[-1]]),
    }
    return sample

In [ ]:
train_data_config = {
    "path":"/content/drive/MyDrive/mipt_2024/embs_p1.csv",
    "past_size":11,
    "emb_cols":[f"e{i}" for i in range(128)],
    "train":True,
    "val_size":0.2,
}
val_data_config = {
    "path":"/content/drive/MyDrive/mipt_2024/embs_p1.csv",
    "past_size":11,
    "emb_cols":[f"e{i}" for i in range(128)],
    "train":False,
    "val_size":0.2,
}
train_dataset = TSDataset(train_data_config)
val_dataset = TSDataset(val_data_config)

In [ ]:
config = PatchTSTConfig(
    num_input_channels=train_dataset[0]["past_values"].shape[1],
    context_length=6,
    patch_length=1,
    patch_stride=1,
    prediction_length=1,
    random_mask_ratio=0.0,
    d_model=train_dataset[0]["past_values"].shape[1],
    num_attention_heads=16,
    num_hidden_layers=3,
    ffn_dim=256,
    dropout=0.2,
    head_dropout=0.2,
    pooling_type=None,
    channel_attention=False,
    scaling="std",
    loss="mse",
    pre_norm=True,
    norm_type="batchnorm",
)
model = PatchTSTForPrediction(config)

In [ ]:
training_args = TrainingArguments(
    output_dir="/content",
    overwrite_output_dir=True,
    # learning_rate=0.001,
    num_train_epochs=100,
    do_eval=True,
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    dataloader_num_workers=num_workers,
    save_strategy="epoch",
    logging_strategy="epoch",
    save_total_limit=3,
    logging_dir="/content",  # Make sure to specify a logging directory
    load_best_model_at_end=True,  # Load the best model when training ends
    metric_for_best_model="eval_loss",  # Metric to monitor for early stopping
    greater_is_better=False,  # For loss
    label_names=["future_values"],
)

# define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# pretrain
trainer.train()


##Predict test

In [ ]:
target_vars = torch.Tensor([
    [0, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0],
    [0, 0, 1, 1],
    [0, 1, 0, 0],
    [0, 1, 0, 1],
    [0, 1, 1, 0],
    [0, 1, 1, 1],
    [1, 0, 0, 0],
    [1, 0, 0, 1],
    [1, 0, 1, 0],
    [1, 0, 1, 1],
    [1, 1, 0, 0],
    [1, 1, 0, 1],
    [1, 1, 1, 0],
    [1, 1, 1, 1],
])
target_embs = clip_model.target_model(target_vars.to(device))

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
data_config = {
    "path":"",
    "past_size":6,
    "train":False,
    "val_size":0.2,
}
dataset = TSDataset(data_config)
dataloader = DataLoader(dataset, batch_size=16, shuffle=False)
model = trainer.model

In [ ]:
embs = None
targets = None
sim = nn.CosineSimilarity()
for batch in tqdm(dataloader):
  for key in batch.keys():
    batch[key] = batch[key].to(device)
  out = model(batch)["future_values"].squeeze(1)
  if embs == None:
    embs = out
  else:
    embs = torch.cat([embs, out], dim=0)
  local_targets = target_vars[(out @ target_embs.T).argmax(dim=1)]
  if targets == None:
    targets = local_targets
  else:
    targets = torch.cat([targets, local_targets], dim=0)